# Libraries

In [11]:
import numpy as np
import pandas as pd

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.algorithims  import *
from pkg.neighborhood import *

from time import time

# Simulates Annealing

In [4]:
seeds = np.array([7054, 1354, 23503, 11268, 58283])

ev = Evaluation()

granularity = 2 # estudiado en local search


A continuación se muestran los parámetros que quedan por ajustar y que ajustaremos en las siguientes celdas

In [13]:
def run_simulated_an(seeds, L, k_max, mu, fi, granularity, ev):
    costs       = np.array([])
    evaluations = np.array([])

    ini = time()

    greedy_cost = 574.5513854177129
    t0 = mu * greedy_cost / (- np.log(fi))
    tf = t0 / (1 + k_max)

    for s in seeds:

        # set evaluation calls counter to 0
        ev.total_calls = 0

        np.random.seed(s)

        solution, cost = simulated_an(
            t0         = t0,
            tf         = tf,
            L          = L, 
            slots      = granularity, 
            evaluation = ev
        )

        costs = np.append(costs, cost)
        evaluations = np.append(evaluations, ev.total_calls)


    fin = time()

    df = pd.DataFrame({
        "L"                 : [L],
        "k_max"             : [k_max],
        "mu"                : [mu],
        "fi"                : [fi],
        "tiempo_ejecucion"  : [fin-ini],
        "media_coste"       : [costs.mean()],
        "std_coste"         : [costs.std()],
        "media_evaluaciones": [evaluations.mean()],
        "std_evaluaciones"  : [evaluations.std()],
        "coste_mejor"       : [costs.min()]
    })

    return df


In [14]:
# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

df = run_simulated_an(
    seeds       = seeds,
    L           = L,
    k_max       = k_max,
    mu          = mu,
    fi          = fi,
    granularity = granularity,
    ev          = ev
)


In [15]:
df

,L,k_max,mu,fi,tiempo_ejecucion,media_coste,std_coste,media_evaluaciones,std_evaluaciones,coste_mejor
0,20,80,0.2,0.2,424.141814,388.57637,13.950863,1641.0,0.0,375.322033


## Ajuste de parametros

Ajuste de mu y fi. Guardamos todos los mu y fi que cumplan la condición en un dataframe 

In [31]:
greedy_solution = np.array([ 7,  9, 18,  8, 11, 18, 11, 12,  8, 13, 13, 24, 11, 18, 20, 19])

n = Neighborhood(greedy_solution,2)
candidata = n.get_neighbor()



cost_actual = ev.evaluate(greedy_solution)
cost_cand = ev.evaluate(candidata)

dif = cost_cand - cost_actual

mus = np.array([])
fis = np.array([])
t = 0


for mu in np.arange(0.1,0.35, 0.05):
    for fi in np.arange(0.1,0.35, 0.05):

        aceptadas = 0
        t +=1

        T = mu * cost_actual / (- np.log(fi))

        for it in range(1000):
            if np.random.rand() < np.exp(-dif/T):
                aceptadas += 1
        
        if aceptadas > 800:
            mus = np.append(mus,mu)
            fis = np.append(fis,fi)

df_mu_fi = pd.DataFrame({
    "mu": mus,
    "fi": fis
})



In [33]:
df_mu_fi

,mu,fi
0,0.10,0.10
1,0.10,0.15
2,0.10,0.20
3,0.10,0.25
4,0.10,0.30
5,0.15,0.10
6,0.15,0.15
7,0.15,0.20
8,0.15,0.25
9,0.15,0.30
